### Training model

In [7]:
import pandas as pd
df = pd.read_csv('../Data/Data_WinPrediction.csv')
#df = pd.read_csv('Data_Changed.csv').drop(['unnamed 0'],axis=1)


print(df)

        level  day_diff mode_game  win  quantity version
0           1       0.0    normal  0.0        31   1.6.0
1           1       0.0    normal  1.0        18   1.6.0
2           2       0.0    normal  1.0        10   1.6.0
3           3       0.0    normal  1.0         9   1.6.0
4           4       0.0    normal  1.0         8   1.6.0
...       ...       ...       ...  ...       ...     ...
243692      4       6.0    normal  1.0        19   1.6.0
243693      6       6.0    normal  1.0         7   1.6.0
243694      5       6.0    normal  1.0        12   1.6.0
243695      1       6.0    normal  1.0        14   1.6.0
243696      3       6.0    normal  1.0         3   1.6.0

[243697 rows x 6 columns]


In [8]:
# One-hot encoding cho các cột phân loại
from sklearn.preprocessing import OneHotEncoder
categorical_cols = ['mode_game', 'version']
encoder = OneHotEncoder(handle_unknown='ignore', sparse_output=False)


encoded_data = encoder.fit_transform(df[categorical_cols])
encoded_df = pd.DataFrame(
    encoded_data, 
    columns=encoder.get_feature_names_out(categorical_cols),
    index=df.index  # Keep the original index!
)


# 2. Drop the original categorical columns from df:
df.drop(columns=categorical_cols, inplace=True)

# 3. Join encoded df back to original df while aligning indices:
df = df.join(encoded_df)

# Verification Step:
print(f"NaN Values in win column before encoding: {df['win'].isna().sum()}")
print(f"NaN Values in win column after encoding: {df['win'].isna().sum()}")

NaN Values in win column before encoding: 0
NaN Values in win column after encoding: 0


In [9]:
# Chia dữ liệu thành tập huấn luyện và tập kiểm tra
from sklearn.model_selection import train_test_split
y = df['win']
print(y.sum())
X = df.drop(columns=['win'])

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)


126284.0


In [10]:
print(X_test)

        level  day_diff  quantity  mode_game_normal  version_1.5.2  \
43080      11       0.0        64               1.0            0.0   
224948     13       3.0        13               1.0            1.0   
180083     13       3.0        19               1.0            1.0   
12562     191       5.0        11               1.0            1.0   
111943     12       0.0        25               1.0            1.0   
...       ...       ...       ...               ...            ...   
240706     54       1.0       169               1.0            0.0   
6830        5       1.0        37               1.0            0.0   
37359       2       0.0         9               1.0            1.0   
223044      7       0.0        14               1.0            1.0   
169399     11       0.0        18               1.0            1.0   

        version_1.6.0  
43080             1.0  
224948            0.0  
180083            0.0  
12562             0.0  
111943            0.0  
...            

In [11]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
from sklearn.preprocessing import OneHotEncoder
from sklearn.tree import export_graphviz
import graphviz

# Xây dựng mô hình Decision Tree
model = DecisionTreeClassifier(random_state=42)  # Thêm random_state để tái lặp kết quả
model.fit(X_train, y_train)

# Dự đoán trên tập kiểm tra
y_pred = model.predict(X_test)

# Đánh giá mô hình
accuracy = accuracy_score(y_test, y_pred)
print(f'Accuracy: {accuracy:.2f}')
print("\nClassification Report:\n")
print(classification_report(y_test, y_pred))
print("\nConfusion Matrix:\n")
print(confusion_matrix(y_test, y_pred))

# Trực quan hóa cây quyết định (nếu cần)
dot_data = export_graphviz(model, out_file=None, feature_names=X.columns, class_names=['Lose', 'Win'], filled=True, rounded=True, special_characters=True)

Accuracy: 0.78

Classification Report:

              precision    recall  f1-score   support

         0.0       0.79      0.74      0.77     23486
         1.0       0.77      0.82      0.80     25254

    accuracy                           0.78     48740
   macro avg       0.78      0.78      0.78     48740
weighted avg       0.78      0.78      0.78     48740


Confusion Matrix:

[[17484  6002]
 [ 4587 20667]]


In [12]:
output = model.predict([[ 1 ,0.0        ,64           ,    1.0    ,        0.0   ,1]])
print(output)

[0.]


/opt/homebrew/lib/python3.11/site-packages/sklearn/base.py:465: UserWarning: X does not have valid feature names, but DecisionTreeClassifier was fitted with feature names
  warnings.warn(
